<a href="https://colab.research.google.com/github/abhi-11nav/Text-Emotion-Detection/blob/main/Text_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
# Importing the necessary libraries 

import pandas as pd
import numpy as np 

In [212]:
# Cloning the github repository 

!git clone https://github.com/abhi-11nav/Text-Emotion-Detection.git

fatal: destination path 'Text-Emotion-Detection' already exists and is not an empty directory.


In [213]:
# Importing data

data = pd.read_csv("/content/Text-Emotion-Detection/tweet_emotions.csv")

In [214]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [215]:
# Let us drop the tweet id

data.drop("tweet_id", axis=1, inplace=True)

In [216]:
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [217]:
# Let us check if the tweet has any missing values 

data.isna().any()

sentiment    False
content      False
dtype: bool

No missing values

In [218]:
# Let us check the number of categories in sentiment variable

data['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

The data seems imbalanced. Let us deal with it after a bit

In [219]:
# Let us look at the sentences

data['content'][0]

'@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =['

In [220]:
data['content'][1]

'Layin n bed with a headache  ughhhh...waitin on your call...'

Text Preprocessing

In [221]:
# Importing libraries

import re 

import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [222]:
def text_preprocess(dataset,list_name):
  
  for i in range(dataset.shape[0]):
    list_name.append(re.sub('[^a-zA-Z]',' ',str(dataset.iloc[i,1])))

  print("Number and other symbols eliminated from the text")

  # String spacing 
  for x in range(len(list_name)):
    list_name[x] = " ".join(y for y in str(list_name[x]).split()).lower()

  print("Text reorganized and converted to small letter")
  
  for index in range(len(list_name)):
    temp_list= []
    # Lemmatization
    for word in list_name[index].split():
      if word not in stopwords.words('english'):
        temp_list.append(word)
    
    list_name[index] = " ".join(lemmatizer.lemmatize(words) for words in temp_list )

In [223]:
sentences = []

text_preprocess(data,sentences)

Number and other symbols eliminated from the text
Text reorganized and converted to small letter


In [224]:
len(sentences)

40000

Text preprocessing done

Converting text to vectors 

Word2vec

In [225]:
# Importing necessary libraries

import gensim

from gensim.models import Word2Vec

from tqdm import tqdm

from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [226]:
# Words list

words_list = []

# looping through to append words
for index in range(len(sentences)):
  words_list.append(nltk.word_tokenize(sentences[index]))

print(len(words_list)," length of sentences")

40000  length of sentences


In [227]:
model = gensim.models.Word2Vec(words_list, window=5, min_count = 2)

In [229]:
# Empty list 
X = []

# Looping though words
for words in tqdm(words_list):
  X.append(np.mean([model.wv[word] for word in words if word in model.wv.index2word], axis=0))

100%|██████████| 40000/40000 [00:29<00:00, 1337.43it/s]


In [230]:
y = data["sentiment"]

In [231]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

In [232]:
y = le.fit_transform(y)

Train test split

In [234]:
X = [val.tolist() for val in X]

In [239]:
X = np.array(X)

In [240]:
X.shape

(40000,)

In [241]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.1,random_state=101)

In [242]:
X[:5]

array([list([-0.3065258264541626, 0.12740002572536469, -0.12171502411365509, -0.0970512181520462, -0.4912981390953064, 0.06366962194442749, 0.015605923719704151, 0.47017550468444824, -0.2934848368167877, 0.2072802484035492, -0.0023908233270049095, 0.48067429661750793, -0.3952907621860504, -0.19955764710903168, -0.6293414831161499, 0.22451119124889374, -0.14762906730175018, 0.05015379190444946, 0.1730164736509323, 0.653520405292511, 0.17127713561058044, -0.11119307577610016, 0.1938396394252777, -0.15994854271411896, 0.1648135632276535, 0.2437869757413864, 0.20727795362472534, 0.20484641194343567, 0.399741530418396, 0.12535996735095978, 0.2687130272388458, -0.016761435195803642, -0.2709988057613373, -0.31958699226379395, -0.13294976949691772, 0.4309920072555542, -0.5058873891830444, -0.08081002533435822, 0.19043804705142975, -0.12645652890205383, -0.26898494362831116, -0.3066118359565735, -0.3582933247089386, -0.5967003107070923, -0.3600606322288513, -0.2959582805633545, -0.1013002097606

In [243]:
y.shape

(40000,)

In [244]:
np.expand_dims(X[0],axis=-1).shape

(100, 1)

.

## Bi-directional LSTM RNN 

Implementing Bi-directional Long short term Memory recurrent neural network 

In [245]:
# Importing the necessary libraries

import tensorflow 
from tensorflow import keras

from keras.layers import Dense, Flatten, Input, LSTM, Bidirectional, Embedding, Dropout
from keras.models import Model, Sequential

In [246]:
# Unique words

unique_word_list = []

for index in range(len(sentences)):
  [unique_word_list.append(w) for w in sentences[index].split()]


unique_words = list(set(unique_word_list))

print(len(unique_words))

42763


Sequential API

In [247]:
classes = len(data['sentiment'].unique())

print(classes)

13


In [248]:
model = Sequential()

In [249]:
# Model 

model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(classes, activation='softmax'))

In [250]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [251]:
model.summary()

ValueError: ignored

In [252]:
train_X.dtype

dtype('O')

In [253]:
train_X = np.array([np.array(val).astype('float64') for val in train_X])
train_y = np.array([np.array(val).astype('float64') for val in train_y])
test_X = np.array([np.array(val).astype('float64') for val in test_X])
test_y = np.array([np.array(val).astype('float64') for val in test_y])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [255]:
model.fit(train_X, train_y, epochs=10)

ValueError: ignored